# Vérification du Lemme de Localisation Spectrale

**Objectif** : Vérifier numériquement que sur ℂ³/ℤ₂ avec métrique Eguchi-Hanson:

$$\lambda_1 = \frac{1}{4} \quad \text{indépendamment de } \varepsilon$$

---

In [ ]:
import numpy as np
from scipy.integrate import solve_bvp, quad
from scipy.linalg import eigh_tridiagonal, eigh
from scipy.sparse import diags
from scipy.sparse.linalg import eigsh
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

print("Notebook ready for Eguchi-Hanson spectral analysis")

## Étape 1 : Problème de Sturm-Liouville

Résoudre :
$$-f''(x) + V_{\text{eff}}(x) f(x) = \lambda f(x)$$

avec le potentiel effectif :
$$V_{\text{eff}}(x) = \frac{1}{x^2 + \varepsilon^4}$$

In [ ]:
def sturm_liouville_eigenvalues(epsilon, n_points=1000, x_max=50, n_eigenvalues=5):
    """
    Solve the Sturm-Liouville problem:
    -f''(x) + V(x)f(x) = λf(x)
    
    with V(x) = 1/(x² + ε⁴)
    
    Uses finite difference discretization.
    """
    # Grid
    x = np.linspace(0.001, x_max, n_points)  # Avoid x=0 singularity
    dx = x[1] - x[0]
    
    # Potential
    eps4 = epsilon**4
    V = 1.0 / (x**2 + eps4)
    
    # Build tridiagonal Hamiltonian matrix
    # H = -d²/dx² + V(x)
    # Second derivative: f'' ≈ (f_{i+1} - 2f_i + f_{i-1}) / dx²
    
    # Diagonal: 2/dx² + V(x)
    diagonal = 2.0 / dx**2 + V
    
    # Off-diagonal: -1/dx²
    off_diagonal = -np.ones(n_points - 1) / dx**2
    
    # Solve eigenvalue problem
    eigenvalues, eigenvectors = eigh_tridiagonal(diagonal, off_diagonal)
    
    return eigenvalues[:n_eigenvalues], eigenvectors[:, :n_eigenvalues], x, V


# Test for multiple ε values
epsilon_values = [0.01, 0.05, 0.1, 0.2, 0.5, 1.0, 2.0, 5.0]
results_sl = []

print("Sturm-Liouville Results:")
print("="*50)
print(f"{'ε':>8} | {'λ₁':>10} | {'λ₂':>10} | {'λ₃':>10}")
print("-"*50)

for eps in epsilon_values:
    eigenvalues, eigenvectors, x, V = sturm_liouville_eigenvalues(eps)
    results_sl.append({
        'epsilon': eps,
        'lambda_1': eigenvalues[0],
        'lambda_2': eigenvalues[1],
        'lambda_3': eigenvalues[2],
        'eigenvalues': eigenvalues,
        'x': x,
        'V': V,
        'eigenvectors': eigenvectors
    })
    print(f"{eps:>8.2f} | {eigenvalues[0]:>10.6f} | {eigenvalues[1]:>10.6f} | {eigenvalues[2]:>10.6f}")

print("-"*50)
lambda1_values = [r['lambda_1'] for r in results_sl]
print(f"{'Mean λ₁':>8} | {np.mean(lambda1_values):>10.6f}")
print(f"{'Std λ₁':>8} | {np.std(lambda1_values):>10.6f}")
print(f"{'Target':>8} | {0.25:>10.6f}")

In [ ]:
# Plot λ₁ vs ε
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: λ₁ vs ε
ax = axes[0, 0]
ax.semilogx(epsilon_values, lambda1_values, 'bo-', markersize=10, linewidth=2)
ax.axhline(y=0.25, color='r', linestyle='--', linewidth=2, label='Target: λ₁ = 1/4')
ax.set_xlabel('ε')
ax.set_ylabel('λ₁')
ax.set_title('Premier valeur propre vs ε (Sturm-Liouville)')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_ylim([0, 0.5])

# Plot 2: Potential V(x) for different ε
ax = axes[0, 1]
for r in results_sl[::2]:  # Every other ε
    ax.plot(r['x'][:200], r['V'][:200], label=f"ε = {r['epsilon']}")
ax.set_xlabel('x')
ax.set_ylabel('V(x) = 1/(x² + ε⁴)')
ax.set_title('Potentiel effectif')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_xlim([0, 5])

# Plot 3: First eigenfunction for different ε
ax = axes[1, 0]
for r in results_sl[::2]:
    f1 = r['eigenvectors'][:, 0]
    f1 = f1 / np.max(np.abs(f1))  # Normalize
    ax.plot(r['x'][:300], f1[:300], label=f"ε = {r['epsilon']}")
ax.set_xlabel('x')
ax.set_ylabel('f₁(x) (normalized)')
ax.set_title('Première fonction propre')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 4: Deviation from 1/4
ax = axes[1, 1]
deviations = [(l - 0.25) / 0.25 * 100 for l in lambda1_values]
ax.bar(range(len(epsilon_values)), deviations, color='steelblue', alpha=0.7)
ax.set_xticks(range(len(epsilon_values)))
ax.set_xticklabels([f"{e}" for e in epsilon_values])
ax.set_xlabel('ε')
ax.set_ylabel('Déviation de 1/4 (%)')
ax.set_title('Écart relatif par rapport à λ₁ = 1/4')
ax.axhline(y=0, color='r', linestyle='--')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('sturm_liouville_results.png', dpi=150)
plt.show()

## Étape 2 : Vérification de la Fonction Propre Explicite

Kimi propose :
$$f_1(x) = \frac{x}{\sqrt{x^2 + \varepsilon^4}}$$

Vérifions que $\Delta f_1 = \frac{1}{4} f_1$

In [ ]:
def verify_eigenfunction(epsilon, x):
    """
    Verify that f₁(x) = x/√(x² + ε⁴) satisfies
    -f₁''(x) + V(x)f₁(x) = λ f₁(x)
    
    Returns the effective λ at each point.
    """
    eps4 = epsilon**4
    
    # f₁(x) = x / √(x² + ε⁴)
    denom = np.sqrt(x**2 + eps4)
    f1 = x / denom
    
    # First derivative: f₁'(x)
    # d/dx [x / √(x² + ε⁴)] = √(x² + ε⁴)⁻¹ - x² (x² + ε⁴)^{-3/2}
    #                       = (x² + ε⁴ - x²) / (x² + ε⁴)^{3/2}
    #                       = ε⁴ / (x² + ε⁴)^{3/2}
    f1_prime = eps4 / (x**2 + eps4)**1.5
    
    # Second derivative: f₁''(x)
    # d/dx [ε⁴ / (x² + ε⁴)^{3/2}] = -3 ε⁴ x / (x² + ε⁴)^{5/2}
    f1_double_prime = -3 * eps4 * x / (x**2 + eps4)**2.5
    
    # Potential
    V = 1.0 / (x**2 + eps4)
    
    # Compute -f₁'' + V f₁
    Hf1 = -f1_double_prime + V * f1
    
    # Effective eigenvalue: λ = (H f₁) / f₁
    # Avoid division by zero near x=0
    mask = np.abs(f1) > 1e-10
    lambda_eff = np.zeros_like(x)
    lambda_eff[mask] = Hf1[mask] / f1[mask]
    
    return f1, f1_prime, f1_double_prime, Hf1, lambda_eff, V


# Test for different ε
print("Eigenfunction Verification:")
print("="*60)

x_test = np.linspace(0.1, 20, 1000)

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

for i, eps in enumerate([0.1, 0.5, 1.0, 2.0]):
    f1, f1p, f1pp, Hf1, lambda_eff, V = verify_eigenfunction(eps, x_test)
    
    ax = axes[i // 2, i % 2]
    
    # Plot effective λ
    ax.plot(x_test, lambda_eff, 'b-', linewidth=2, label='λ_eff(x) = (Hf₁)/f₁')
    ax.axhline(y=0.25, color='r', linestyle='--', linewidth=2, label='Target: 1/4')
    
    # Statistics
    mean_lambda = np.mean(lambda_eff[100:])  # Skip near-boundary region
    std_lambda = np.std(lambda_eff[100:])
    
    ax.set_xlabel('x')
    ax.set_ylabel('λ_eff')
    ax.set_title(f'ε = {eps}: λ_eff = {mean_lambda:.4f} ± {std_lambda:.4f}')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.set_ylim([0, 0.5])
    
    print(f"ε = {eps}: λ_eff (mean) = {mean_lambda:.6f}, std = {std_lambda:.6f}")

plt.tight_layout()
plt.savefig('eigenfunction_verification.png', dpi=150)
plt.show()

## Étape 3 : Laplacien sur Métrique Eguchi-Hanson Complète

La métrique Eguchi-Hanson en coordonnées $(r, \theta, \phi, \psi)$ :

$$g_{EH} = \left(1 - \frac{\varepsilon^4}{r^4}\right)^{-1} dr^2 + r^2 (\sigma_1^2 + \sigma_2^2) + r^2 \left(1 - \frac{\varepsilon^4}{r^4}\right) \sigma_3^2$$

où $\sigma_i$ sont les formes de Maurer-Cartan sur $S^3$.

In [ ]:
def eguchi_hanson_laplacian_radial(epsilon, n_points=500, r_max=30):
    """
    Compute eigenvalues of the radial part of the Laplacian
    on Eguchi-Hanson metric.
    
    For the scalar Laplacian on EH, with angular momentum ℓ = 0,
    the radial equation is:
    
    -1/(r³√h) d/dr(r³√h df/dr) = λf
    
    where h(r) = 1 - ε⁴/r⁴
    """
    eps4 = epsilon**4
    
    # Grid: r from ε (bolt) to r_max
    r = np.linspace(epsilon + 0.001, r_max, n_points)
    dr = r[1] - r[0]
    
    # Metric function h(r) = 1 - ε⁴/r⁴
    h = 1 - eps4 / r**4
    h = np.maximum(h, 1e-10)  # Regularize near bolt
    sqrt_h = np.sqrt(h)
    
    # Volume factor: r³ √h
    vol = r**3 * sqrt_h
    
    # Build Laplacian matrix using finite differences
    # Δf = -1/vol d/dr(vol h df/dr)
    
    n = len(r)
    L = np.zeros((n, n))
    
    for i in range(1, n-1):
        # Coefficients for second-order central difference
        # with variable coefficients
        
        # A(r) = vol(r) * h(r) = r³ h^{3/2}
        A_plus = 0.5 * (vol[i] * h[i] + vol[i+1] * h[i+1])
        A_minus = 0.5 * (vol[i] * h[i] + vol[i-1] * h[i-1])
        
        L[i, i+1] = -A_plus / (vol[i] * dr**2)
        L[i, i-1] = -A_minus / (vol[i] * dr**2)
        L[i, i] = (A_plus + A_minus) / (vol[i] * dr**2)
    
    # Boundary conditions: Neumann at bolt, Dirichlet at infinity
    L[0, 0] = L[1, 1]
    L[0, 1] = L[1, 2]
    L[-1, -1] = 1
    L[-1, -2] = 0
    
    # Solve eigenvalue problem
    eigenvalues, eigenvectors = np.linalg.eigh(L)
    
    # Filter positive eigenvalues
    pos_mask = eigenvalues > 1e-6
    eigenvalues = eigenvalues[pos_mask]
    
    return eigenvalues[:10], r, h


# Test Eguchi-Hanson Laplacian
print("\nEguchi-Hanson Radial Laplacian:")
print("="*50)

eh_results = []
for eps in epsilon_values:
    try:
        eigenvalues, r, h = eguchi_hanson_laplacian_radial(eps)
        lambda1 = eigenvalues[0] if len(eigenvalues) > 0 else np.nan
        eh_results.append({'epsilon': eps, 'lambda_1': lambda1, 'eigenvalues': eigenvalues})
        print(f"ε = {eps:>5.2f}: λ₁ = {lambda1:.6f}")
    except Exception as e:
        print(f"ε = {eps:>5.2f}: Error - {e}")
        eh_results.append({'epsilon': eps, 'lambda_1': np.nan})

## Étape 4 : Test d'Indépendance en ε

Le **miracle** attendu : λ₁(ε) = 1/4 pour tout ε.

In [ ]:
# High-resolution scan of ε
epsilon_fine = np.logspace(-2, 1, 30)  # 0.01 to 10

lambda1_sl = []  # Sturm-Liouville results
lambda1_eh = []  # Eguchi-Hanson results

print("High-resolution ε scan:")
print("="*60)

for eps in epsilon_fine:
    # Sturm-Liouville
    ev_sl, _, _, _ = sturm_liouville_eigenvalues(eps, n_points=2000, x_max=100)
    lambda1_sl.append(ev_sl[0])
    
    # Eguchi-Hanson
    try:
        ev_eh, _, _ = eguchi_hanson_laplacian_radial(eps, n_points=1000, r_max=50)
        lambda1_eh.append(ev_eh[0] if len(ev_eh) > 0 else np.nan)
    except:
        lambda1_eh.append(np.nan)

lambda1_sl = np.array(lambda1_sl)
lambda1_eh = np.array(lambda1_eh)

# Statistics
print(f"\nSturm-Liouville:")
print(f"  Mean λ₁ = {np.mean(lambda1_sl):.6f}")
print(f"  Std λ₁  = {np.std(lambda1_sl):.6f}")
print(f"  Min λ₁  = {np.min(lambda1_sl):.6f}")
print(f"  Max λ₁  = {np.max(lambda1_sl):.6f}")

eh_valid = lambda1_eh[~np.isnan(lambda1_eh)]
if len(eh_valid) > 0:
    print(f"\nEguchi-Hanson:")
    print(f"  Mean λ₁ = {np.mean(eh_valid):.6f}")
    print(f"  Std λ₁  = {np.std(eh_valid):.6f}")

In [ ]:
# Final visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: λ₁(ε) for Sturm-Liouville
ax = axes[0, 0]
ax.semilogx(epsilon_fine, lambda1_sl, 'b.-', markersize=8, linewidth=1.5, label='Sturm-Liouville')
ax.axhline(y=0.25, color='r', linestyle='--', linewidth=2, label='Target: 1/4')
ax.fill_between(epsilon_fine, 0.24, 0.26, alpha=0.2, color='green', label='±4% band')
ax.set_xlabel('ε')
ax.set_ylabel('λ₁')
ax.set_title('Sturm-Liouville: λ₁ vs ε')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_ylim([0.15, 0.35])

# Plot 2: Relative deviation
ax = axes[0, 1]
deviation_sl = (lambda1_sl - 0.25) / 0.25 * 100
ax.semilogx(epsilon_fine, deviation_sl, 'b.-', markersize=8, linewidth=1.5)
ax.axhline(y=0, color='r', linestyle='--', linewidth=2)
ax.fill_between(epsilon_fine, -5, 5, alpha=0.2, color='green', label='±5% band')
ax.set_xlabel('ε')
ax.set_ylabel('Déviation (%)')
ax.set_title('Écart relatif par rapport à 1/4')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 3: Histogram of λ₁ values
ax = axes[1, 0]
ax.hist(lambda1_sl, bins=20, color='steelblue', alpha=0.7, edgecolor='black')
ax.axvline(x=0.25, color='r', linestyle='--', linewidth=2, label='Target: 1/4')
ax.axvline(x=np.mean(lambda1_sl), color='green', linestyle='-', linewidth=2, label=f'Mean: {np.mean(lambda1_sl):.4f}')
ax.set_xlabel('λ₁')
ax.set_ylabel('Count')
ax.set_title('Distribution de λ₁ (Sturm-Liouville)')
ax.legend()

# Plot 4: Summary box
ax = axes[1, 1]
ax.axis('off')

# Check if miracle is confirmed
mean_dev = np.abs(np.mean(lambda1_sl) - 0.25) / 0.25 * 100
max_dev = np.max(np.abs(lambda1_sl - 0.25)) / 0.25 * 100
variance = np.var(lambda1_sl)

miracle_confirmed = mean_dev < 5 and max_dev < 10

summary_text = f"""
╔══════════════════════════════════════════════════════════╗
║         RÉSUMÉ : LEMME DE LOCALISATION SPECTRALE         ║
╠══════════════════════════════════════════════════════════╣
║                                                          ║
║  Potentiel: V(x) = 1/(x² + ε⁴)                          ║
║  Valeur cible: λ₁ = 1/4 = 0.25                          ║
║                                                          ║
║  RÉSULTATS (Sturm-Liouville):                           ║
║  ─────────────────────────────                           ║
║  λ₁ moyen:      {np.mean(lambda1_sl):.6f}                        ║
║  Écart-type:    {np.std(lambda1_sl):.6f}                        ║
║  Dév. moyenne:  {mean_dev:.2f}%                               ║
║  Dév. max:      {max_dev:.2f}%                               ║
║  Variance:      {variance:.2e}                        ║
║                                                          ║
║  VERDICT: {'✅ MIRACLE CONFIRMÉ!' if miracle_confirmed else '❌ MIRACLE NON CONFIRMÉ'}                      ║
║                                                          ║
╚══════════════════════════════════════════════════════════╝
"""

ax.text(0.1, 0.5, summary_text, fontsize=11, fontfamily='monospace',
        verticalalignment='center', transform=ax.transAxes,
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

plt.tight_layout()
plt.savefig('spectral_localization_lemma.png', dpi=150)
plt.show()

print("\n" + "="*60)
print(summary_text)

## Étape 5 : Export des Résultats

In [ ]:
import json

# Export results
results = {
    'target_lambda1': 0.25,
    'sturm_liouville': {
        'epsilon_values': epsilon_fine.tolist(),
        'lambda1_values': lambda1_sl.tolist(),
        'mean': float(np.mean(lambda1_sl)),
        'std': float(np.std(lambda1_sl)),
        'variance': float(np.var(lambda1_sl)),
        'mean_deviation_percent': float(mean_dev),
        'max_deviation_percent': float(max_dev),
    },
    'miracle_confirmed': miracle_confirmed,
    'conclusion': 'λ₁ = 1/4 independent of ε' if miracle_confirmed else 'λ₁ depends on ε'
}

with open('spectral_localization_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print("Results saved to spectral_localization_results.json")
print("Plots saved to:")
print("  - sturm_liouville_results.png")
print("  - eigenfunction_verification.png")
print("  - spectral_localization_lemma.png")

---

## Conclusion

Ce notebook vérifie le lemme de localisation spectrale de Kimi :

**Si le miracle est confirmé** (variance ~ 0) :
- Le potentiel V(x) = 1/(x² + ε⁴) a exactement λ₁ = 1/4
- C'est un cas spécial de potentiel Pöschl-Teller
- La métrique Eguchi-Hanson hérite de cette propriété

**Prochaine étape** : Gluing global pour passer de ℂ²/ℤ₂ local à K₇ compact.